## 2. Semantic Search - Compression

- Sample: https://github.com/Azure/azure-search-vector-samples/blob/main/demo-python/code/vector-quantization-and-storage/vector-quantization-and-storage.ipynb

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    SearchField,  
    VectorSearch,
    SemanticSearch,
    SemanticPrioritizedFields,
    SemanticField,
    HnswAlgorithmConfiguration,
    HnswParameters,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters,
    VectorSearchCompression,
    BinaryQuantizationCompression
)

In [3]:
import os
service_endpoint = os.getenv("AZSCH_ENDPOINT")  
credential = AzureKeyCredential(os.environ["AZSCH_KEY"])

#print(service_endpoint)

api_key = os.getenv("AZURE_OPENAI_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")

In [4]:
# Create a search index
def create_search_index(index_name, model_name='text-embedding-3-large'):
    index_client = SearchIndexClient(
        endpoint=service_endpoint, credential=credential)
    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="proverb", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True),
        SearchableField(name="meaning", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True),
        SearchableField(name="source", type=SearchFieldDataType.String,
                        searchable=False, retrievable=True),
        SearchableField(name="type", type=SearchFieldDataType.String,
                        searchable=False, retrievable=True),
        SearchField(name="vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                    searchable=True, vector_search_dimensions=3072,
                    stored=True,
                    vector_search_profile_name="myHnswProfile")  
    ]

    vector_search = VectorSearch(  
        algorithms=[  
            HnswAlgorithmConfiguration(  
                name="myHnsw",  
                parameters=HnswParameters(  
                    m=4,  
                    ef_construction=400,  
                    ef_search=500,  
                    metric=VectorSearchAlgorithmMetric.COSINE,  
                ),  
            )
        ],  
        profiles=[  
            VectorSearchProfile(  
                name="myHnswProfile",  
                algorithm_configuration_name="myHnsw",
                vectorizer_name="vectorizer",
                compression_name="myCompression"
            )
        ],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="vectorizer",
                kind="azureOpenAI",
                parameters = AzureOpenAIVectorizerParameters(
                    resource_url=azure_endpoint,
                    api_key=api_key,
                    deployment_name=model_name,
                    model_name=model_name
                ),
            )
        ],
        compressions=[
            BinaryQuantizationCompression(compression_name="myCompression", truncation_dimension=1024)
        ]
    )  

    semantic_config = SemanticConfiguration(  
        name="semantic-config",  
        prioritized_fields=SemanticPrioritizedFields(  
            title_field=SemanticField(field_name="proverb"),
            content_fields=[SemanticField(field_name="meaning")]  
        ),  
    )

    # Create the semantic search with the configuration  
    semantic_search = SemanticSearch(configurations=[semantic_config]) 

    # Create the search index
    index = SearchIndex(name=index_name,
                        fields=fields,
                        vector_search=vector_search,
                        semantic_search=semantic_search)
    
    result = index_client.create_or_update_index(index)
    print(f' {result.name} created')

In [5]:
index_name = "korproverb-compression-index"
create_search_index(index_name, 'text-embedding-3-large')

 korproverb-compression-index created


In [6]:
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)

In [7]:
import pandas as pd
from tqdm import tqdm

In [8]:
df = pd.read_pickle('kor_proverbs_embeddings.pkl')
df.head()

,Description,Meaning,Source,Embeddings2,Type,Embeddings3
0,가갸 뒤 자〔뒷다리〕도 모른다,"글자를 모르는 사람을 비유적으로 이르는 말.,사리에 어두운 사람을 놀림조로 이르는 말.",고려대 한국어대사전,"[0.005334234796464443, -0.014505433849990368, ...",속담,"[0.017118478193879128, -0.024258537217974663, ..."
1,가까운 남이 먼 일가보다 낫다,이웃끼리 서로 친하게 지내다 보면 먼 곳에 있는 일가보다 더 친하게 되어 서로 도우...,표준국어대사전,"[-0.007843011058866978, -0.018381033092737198,...",속담,"[-0.028828121721744537, -0.00209252187050879, ..."
2,가까운 데 집은 깎이고 먼 데 절은 비친다,가까운 데 있는 절이나 집은 자잘한 흠도 잘 드러나서 좋지 않아 보이고 먼 곳에 윤...,표준국어대사전,"[-0.0012391135096549988, -0.012439466081559658...",속담,"[-0.0034362655133008957, 0.012371719814836979,..."
3,가까운 무당보다 먼 데 무당이 영하다,흔히 사람은 자신이 잘 알고 가까이 있는 것보다는 잘 모르고 멀리 있는 것을 더 좋...,표준국어대사전,"[0.012985464185476303, -0.0004005177761428058,...",속담,"[0.023922793567180634, -0.011181957088410854, ..."
4,가까운 집 며느리일수록 흉이 많다,늘 가까이 있고 잘 아는 사이일수록 상대편의 결점이 눈에 더 많이 띈다는 말.,표준국어대사전,"[-0.0019731209613382816, -0.02421422116458416,...",속담,"[0.012853839434683323, -0.039608076214790344, ..."


In [9]:
count = 0
batch_size = 20
for i in tqdm(range(0, len(df), batch_size)):
    # set end position of batch
    i_end = min(i+batch_size, len(df))
    
    documents = df[i:i_end].apply(
        lambda row: {'id': str(row.name), 
                     'proverb': row['Description'], 
                     'meaning': row['Meaning'],
                     'source': row['Source'],
                     'type': str(row['Type']),
                     #'vector': generate_embeddings(row['chunk'])
                     'vector': row['Embeddings3']
                    }, axis=1).to_list()
    
    result = search_client.upload_documents(documents)  

100%|██████████| 224/224 [02:03<00:00,  1.81it/s]


In [11]:
from azure.search.documents.models import (
    VectorizableTextQuery,
    VectorQuery,
    VectorizedQuery,
    QueryType,
    QueryCaptionType,
    QueryAnswerType)

In [12]:
def azsch_embed_query(query):
    #vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="vector")
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=10, fields="vector", exhaustive=True)

    results = search_client.search(  
        search_text=None,  
        vector_queries=[vector_query],
        select=["proverb", "meaning", "source", "type"],
        top=5 # for limiting text search
    ) 

    for result in results:  
        print(f"{result['@search.score']:.10f}: {result['proverb']} - {result['meaning']}, {result['type']}")  


In [13]:
res = azsch_embed_query("열심히 노력하면 성공한다는 말")

0.7986877600: 십 년 적공이면 한 가지 성공을 한다 - 무슨 일이든지 오랫동안 꾸준히 노력하면 마침내는 성공하게 됨을 이르는 말., 속담
0.7708128000: 절뚝발이 말이 천리 간다 - 누구나 꾸준히 노력하면 무슨 일이든 이룰 수 있다는 말. (=둔한 말도 열흘 가면 천리를 간다.), 속담
0.7679825400: 드나드는 개가 꿩을 문다 - 꾸준하게 열성적으로 노력하는 사람이 일을 이루고 재물을 얻을 수 있다는 말., 속담
0.7645950000: 쥐도 한몫 보면 락이 있다 - 한길로 전심전력하면 성공할 때가 있음을 비유적으로 이르는 말., 속담
0.7573932400: 정성이 지극하면 돌 위에도 풀이 난다 - 진심으로 정성을 다하면 불가능해 보이는 일도 이루어질 수 있다는 말., 속담
